In [ ]:
# PROMT:
# DO THE CHANGES IN MY CODE BECAUSE LIME DOES NOT WORK PROPERLY FOR SPAN TYPE. I WILL GIVE YOU EXAMPLE
# Question: 'How much was the closing net book amount for software under development in 2019?'
# Predicted: span (confidence: 98.56%) 
# IN THIS QUESTION IT DOES NOT EXPLAIN PROPERLY WHY IT IS SPAN AND DOES NOT TAKEN PARTICULAR WORDS WHICH PREDICTS THAT IT IS A SPAN


import lime
import lime.lime_text
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import numpy as np
import re
from nltk.corpus import stopwords
import nltk

# Download NLTK data
nltk.download('stopwords')

# Initialize stopwords with additional filters
stop_words = set(stopwords.words('english'))
extra_filters = {"?", "in", "me", "the", "was", "for", "what", "how"}
filtered_stop_words = stop_words.union(extra_filters)

# Load model and tokenizer
model_name = "rahul14/span-arithmetic-classification"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
explainer = lime.lime_text.LimeTextExplainer(class_names=["arithmetic", "span"])
model.eval()

def predictor(texts):
    if isinstance(texts, str):
        texts = [texts]
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=32)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    return torch.nn.functional.softmax(outputs.logits, dim=1).cpu().numpy()

def custom_split(text):
    # Define important phrases to protect
    important_phrases = [
        "percentage change",
        "difference between",
        "closing net book amount",
        "software under development",
        "total amount",
        "net book amount"
    ]
    
    # Protect phrases by temporarily replacing spaces
    lower_text = text.lower()
    for phrase in important_phrases:
        if phrase in lower_text:
            start = lower_text.find(phrase)
            original_phrase = text[start:start+len(phrase)]
            text = text.replace(original_phrase, original_phrase.replace(" ", "§"))
    
    # Tokenize while preserving protected phrases
    tokens = re.findall(r"\w+|§\w+§|\$[\d\.]+|\S+", text)
    return [t.replace("§", " ") for t in tokens]

def interpret_question(question):
    def highlight_text(words, explanation):
        highlighted = []
        for word in words:
            weight = next((w for feature, w in explanation if feature.lower() in word.lower()), 0)
            if weight > 0.1:
                highlighted.append(f"[+{weight:.2f}]{word}")
            elif weight < -0.1:
                highlighted.append(f"[{weight:.2f}]{word}")
            else:
                highlighted.append(word)
        return highlighted

    try:
        # Prediction
        pred_probs = predictor([question])[0]
        predicted_class = np.argmax(pred_probs)
        confidence = pred_probs[predicted_class]
        
        print(f"\nQuestion: '{question}'")
        print(f"Predicted: {'arithmetic' if predicted_class == 0 else 'span'} (confidence: {confidence:.2%})")
        
        # Explanation
        exp = explainer.explain_instance(question, predictor, num_features=10, num_samples=500, top_labels=1)
        explanation = get_explanation(exp, predicted_class)
        
        print("\nTop features influencing the prediction:")
        for feature, weight in explanation:
            print(f"{feature:20} {weight:+.4f}")
        
        print("\nText highlighting:")
        print(' '.join(highlight_text(question.split(), explanation)))
        
        return exp
    
    except Exception as e:
        print(f"Error interpreting question: {str(e)}")
        return None

def get_explanation(exp, predicted_class):
    try:
        return exp.as_list(label=predicted_class)
    except KeyError:
        return exp.as_list(label=0) if len(exp.available_labels()) > 0 else []

# Example usage
sample_questions = [
    "What was the percentage change in cost of software under development between 2018 and 2019?",
    "What was the net income(loss) in 2019?",
    "What is the average of Gross profit?", 
    "What is the amount of stated capital outstanding on December 31, 2019?"
]

for question in sample_questions:
    _ = interpret_question(question)
    print("\n" + "="*80 + "\n")



# Question: 'What was the percentage change in cost of software under development between 2018 and 2019?'
# Predicted: arithmetic (confidence: 99.86%)

# Key Features Influencing Prediction:
# Feature                        Weight     Direction
# --------------------------------------------------
# percentage change              +0.3791    ↑ SPAN
# cost                           +0.0776    ↑ SPAN

# Text with Key Features Highlighted:
# What was the [percentage change↑] in [cost↑] of software under development between 2018 and 2019 [?↑]
# ...........................IT GIVES CORRECT ANSWER FOR ONE QUESTION BUT WRONG ANSWER FOR SECOND ONE

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nav6d\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



Question: 'What was the percentage change in cost of software under development between 2018 and 2019?'
Predicted: arithmetic (confidence: 99.86%)

Key Features Influencing Prediction:
Feature                        Weight     Direction
--------------------------------------------------
percentage                     +0.1702    ↑ ARITHMETIC
change                         +0.1663    ↑ ARITHMETIC
cost                           -0.0478    ↓ span
2018                           -0.0411    ↓ span
2019                           -0.0394    ↓ span

Text with Key Features Highlighted:
What was the [percentage↑] [ change↑] in cost of software  under  development between 2018 and 2019 [?↑]



Question: 'What was the net income(loss) in 2019?'
Predicted: span (confidence: 99.91%)

Key Features Influencing Prediction:
Feature                        Weight     Direction
--------------------------------------------------
net                            +0.0106    ↑ SPAN
2019                           